In [2]:
import string
import pandas as pd
import numpy as np
import re

In [3]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
#Opening text file with correct Journal Titles in it

lines = [] 
with open ('J_Medline.txt', 'rt') as in_file:
    for line in in_file:
        lines.append(line)


In [4]:
journallines = []


In [5]:
#Real code to strip lines with journal title 

for x in lines:
    if x.startswith('JournalTitle: '):
        x = x.rstrip()
        x = x.replace('JournalTitle: ', '')
        journallines.append(x)
        

In [6]:
#It looks like it worked!

len(journallines)
print(len(journallines))
journallines

30346


["AADE editors' journal",
 'AANA journal',
 'AARN news letter',
 '[Nihon shika hyoron] The Nippon dental review',
 'Academia Peruana de Cirugia',
 'Academy review of the California Academy of Periodontology, United States Section, ARPA Internationale',
 'Acarologia',
 'Acquisitions medicales recentes',
 'Activitas nervosa superior',
 'Acta allergologica',
 'Acta anaesthesiologica',
 'Acta anaesthesiologica Belgica',
 'Acta anaesthesiologica Scandinavica',
 'Acta anaesthesiologica Scandinavica. Supplementum',
 'Acta anatomica',
 'Kaibogaku zasshi. Journal of anatomy',
 'Acta biochimica et biophysica; Academiae Scientiarum Hungaricae',
 'Acta biochimica Polonica',
 'Across the board',
 'Acta biologica Academiae Scientiarum Hungaricae',
 'Academy of management review. Academy of Management',
 'Acta biochimica et biophysica Hungarica',
 'Acta biologiae experimentalis',
 'Academy of Management journal. Academy of Management',
 'Acta Belgica. Medica physica : organe officiel de la Societe ro

In [7]:
#######################################################
#### Getting the DF ###################################
#######################################################

df = pd.read_csv('Welcome.csv', encoding = "latin1")

In [8]:
df['Cleaned']=0
dfclean = df

In [10]:
dfclean.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged),Cleaned
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00,0
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04,0
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56,0
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64,0
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88,0


In [11]:
dfclean['Journal title']=(dfclean['Journal title'].replace('[1]','one', regex=True ))

In [12]:
#######################################################################################
# Algorithm that compares the journal titles within the database with the list of real journal titles found on the website.
# It uses fuzzywuzzy to compare the strings and chooses the best comparison, if no good comparisons are 
# found then nothing happens. There is a cleaned column, 0 represents the title hasnt been changed
# 1 represents a perfect match, 2 represents a 90% or higher match and the title was changed
########################################################################################


#Iterate through rows of dirty database
for index, row in df.iterrows():
    q = 0
    lastq=0
    print('------------------------------------------------\nID: ' + str(index))

    # Make sure it is a string
    
    if type(row['Journal title']) == str:

        # Iterate through titles from the website
        for k in journallines:
            
            # Make both titles being compared lowercased. Also, remove 'the'
            dbname = row['Journal title'].lower()
            webname = k.lower()
            dbname=dbname.replace('the','')
            webname=webname.replace('the','')
            
            q=fuzz.ratio(webname, dbname) #Check ratio

        
        # Change name and switch to next row of dirty dataframe if there is a perfect match
            if q == 100: 
                dfclean.at[index, 'Journal title'] = k
                dfclean.at[index, 'Cleaned'] = 1
                print(' Perfect Match! ' + k)
                break
                  
        
        # Check if the ratio is good enough to change it
            if q > 90 and q >= lastq:
                dfclean.at[index, 'Cleaned'] = 2
                lastq = q
                print(' Score:' + str(lastq) + '\nOG title: '+ row['Journal title'] + ' ||||| Possible Title: ' + k)
                dfclean.at[index, 'Journal title'] = k                
                            

                    
                
#Save it after compile        
dfclean.to_csv('Cleaned.csv')

------------------------------------------------
ID: 0
 Perfect Match! Psychological medicine
------------------------------------------------
ID: 1
 Perfect Match! Biomacromolecules
------------------------------------------------
ID: 2
------------------------------------------------
ID: 3
------------------------------------------------
ID: 4
------------------------------------------------
ID: 5
 Perfect Match! Journal of medicinal chemistry
------------------------------------------------
ID: 6
 Perfect Match! Journal of proteome research
------------------------------------------------
ID: 7
------------------------------------------------
ID: 8
 Perfect Match! ACS chemical biology
------------------------------------------------
ID: 9
 Perfect Match! ACS chemical biology
------------------------------------------------
ID: 10
 Perfect Match! Journal of chemical information and modeling
------------------------------------------------
ID: 11
 Perfect Match! Journal of medicinal c

------------------------------------------------
ID: 69
 Score:98
OG title: Journal of Biological Chemistry ||||| Possible Title: The Journal of biological chemistry
------------------------------------------------
ID: 70
 Score:98
OG title: Journal of Biological Chemistry ||||| Possible Title: The Journal of biological chemistry
------------------------------------------------
ID: 71
 Score:98
OG title: Journal of Biological Chemistry ||||| Possible Title: The Journal of biological chemistry
------------------------------------------------
ID: 72
 Score:98
OG title: Journal of Biological Chemistry ||||| Possible Title: The Journal of biological chemistry
------------------------------------------------
ID: 73
 Score:98
OG title: JOURNAL OF BIOLOGICAL CHEMISTRY ||||| Possible Title: The Journal of biological chemistry
------------------------------------------------
ID: 74
 Score:98
OG title: Journal of Biological Chemistry ||||| Possible Title: The Journal of biological chemistry
----

------------------------------------------------
ID: 134
 Score:98
OG title: Journal of Biological Chemistry ||||| Possible Title: The Journal of biological chemistry
------------------------------------------------
ID: 135
 Score:98
OG title: Journal of Biological Chemistry ||||| Possible Title: The Journal of biological chemistry
------------------------------------------------
ID: 136
 Score:98
OG title: Journal of Biological Chemistry ||||| Possible Title: The Journal of biological chemistry
------------------------------------------------
ID: 137
 Score:98
OG title: JOURNAL OF BIOLOGICAL CHEMISTRY ||||| Possible Title: The Journal of biological chemistry
------------------------------------------------
ID: 138
 Score:98
OG title: Journal of Biological Chemistry ||||| Possible Title: The Journal of biological chemistry
------------------------------------------------
ID: 139
 Score:98
OG title: Journal of Biological Chemistry ||||| Possible Title: The Journal of biological chemistr

 Perfect Match! Genome medicine
------------------------------------------------
ID: 206
 Perfect Match! International journal for equity in health
------------------------------------------------
ID: 207
 Score:99
OG title: International Journal of Behavioral Nutrition and Physical Activity ||||| Possible Title: The international journal of behavioral nutrition and physical activity
------------------------------------------------
ID: 208
 Score:99
OG title: International Journal of Behavioural Nutrition and Physical Activity ||||| Possible Title: The international journal of behavioral nutrition and physical activity
------------------------------------------------
ID: 209
 Perfect Match! International journal of health geographics
------------------------------------------------
ID: 210
 Perfect Match! Journal of cheminformatics
------------------------------------------------
ID: 211
 Perfect Match! Journal of neuroinflammation
------------------------------------------------
ID: 2

------------------------------------------------
ID: 287
 Score:97
OG title: British Journal of Opthalmology ||||| Possible Title: The British journal of ophthalmology
------------------------------------------------
ID: 288
 Perfect Match! British medical journal
------------------------------------------------
ID: 289
 Perfect Match! Frontline gastroenterology
------------------------------------------------
ID: 290
 Perfect Match! Frontline gastroenterology
------------------------------------------------
ID: 291
 Perfect Match! Frontline gastroenterology
------------------------------------------------
ID: 292
 Perfect Match! Journal of clinical pathology
------------------------------------------------
ID: 293
 Score:94
OG title: Journal of Epidemiology & Community Health  ||||| Possible Title: Journal of epidemiology and community health
 Score:94
OG title: Journal of Epidemiology & Community Health  ||||| Possible Title: Journal of epidemiology and community health
-------------

...

------------------------------------------------
ID: 1738
------------------------------------------------
ID: 1739
 Perfect Match! Cognitive processing
------------------------------------------------
ID: 1740
 Perfect Match! Cognitive therapy and research
------------------------------------------------
ID: 1741
 Perfect Match! Current atherosclerosis reports
------------------------------------------------
ID: 1742
 Perfect Match! Current atherosclerosis reports
------------------------------------------------
ID: 1743
 Score:99
OG title: Current Obstetrics and Gynecology Reports  ||||| Possible Title: Current obstetrics and gynecology reports
------------------------------------------------
ID: 1744
 Perfect Match! Development genes and evolution
------------------------------------------------
ID: 1745
 Perfect Match! Diabetologia
------------------------------------------------
ID: 1746
 Perfect Match! Diabetologia
------------------------------------------------
ID: 1747
 Perfec

 Perfect Match! Journal of the American Society for Mass Spectrometry
------------------------------------------------
ID: 1808
 Perfect Match! Age
------------------------------------------------
ID: 1809
 Perfect Match! Biomolecular NMR assignments
------------------------------------------------
ID: 1810
------------------------------------------------
ID: 1811
 Perfect Match! AIDS care
------------------------------------------------
ID: 1812
 Perfect Match! AIDS care
------------------------------------------------
ID: 1813
 Perfect Match! AIDS care
------------------------------------------------
ID: 1814
 Perfect Match! Annals of science
------------------------------------------------
ID: 1815
 Perfect Match! Canadian journal of African studies
------------------------------------------------
ID: 1816
------------------------------------------------
ID: 1817
------------------------------------------------
ID: 1818
 Perfect Match! European journal of English studies
-----------

------------------------------------------------
ID: 1889
 Score:91
OG title: Arthritis & Rheumatism ||||| Possible Title: Arthritis and rheumatism
------------------------------------------------
ID: 1890
 Score:91
OG title: Arthritis & Rheumatism ||||| Possible Title: Arthritis and rheumatism
------------------------------------------------
ID: 1891
 Perfect Match! Arthritis and rheumatism
------------------------------------------------
ID: 1892
 Perfect Match! Arthritis and rheumatism
------------------------------------------------
ID: 1893
 Perfect Match! Bioethics
------------------------------------------------
ID: 1894
 Perfect Match! Bioethics
------------------------------------------------
ID: 1895
 Perfect Match! Bioethics
------------------------------------------------
ID: 1896
 Perfect Match! Biopolymers
------------------------------------------------
ID: 1897
 Score:98
OG title: Birth Defects Research Part A: Clinical and Molecular Teratology ||||| Possible Title: Bir

------------------------------------------------
ID: 1969
 Perfect Match! Journal of the history of the behavioral sciences
------------------------------------------------
ID: 1970
 Score:96
OG title: Journal of the Royal Statistical Society: Series C (Applied Statistics) ||||| Possible Title: Journal of the Royal Statistical Society. Series C, Applied statistics
------------------------------------------------
ID: 1971
 Perfect Match! Magnetic resonance in medicine
------------------------------------------------
ID: 1972
 Score:98
OG title: Magnetic Resonance in Medicine  ||||| Possible Title: Magnetic resonance in medicine
------------------------------------------------
ID: 1973
 Perfect Match! Molecular microbiology
------------------------------------------------
ID: 1974
 Perfect Match! Molecular microbiology
------------------------------------------------
ID: 1975
------------------------------------------------
ID: 1976
------------------------------------------------
ID: 19

------------------------------------------------
ID: 2063
------------------------------------------------
ID: 2064
------------------------------------------------
ID: 2065
 Perfect Match! Depression and anxiety
------------------------------------------------
ID: 2066
------------------------------------------------
ID: 2067
------------------------------------------------
ID: 2068
 Score:98
OG title: Economic History Review ||||| Possible Title: The Economic history review
------------------------------------------------
ID: 2069
 Perfect Match! European journal of immunology
------------------------------------------------
ID: 2070
 Perfect Match! European journal of immunology
------------------------------------------------
ID: 2071
 Score:96
OG title: FEBS Journal ||||| Possible Title: The FEBS journal
------------------------------------------------
ID: 2072
 Perfect Match! FEMS microbiology letters
------------------------------------------------
ID: 2073
 Perfect Match! FEMS 

In [176]:
#Change the name cost column to look better

dfclean.rename(columns={'COST (£) charged to Wellcome (inc VAT when charged)':'Cost'}, inplace = True) 
dfcleancost = dfclean

In [177]:
# Remove pound symbol and convert to float so we are able to do calculations 

for index, row in dfclean.iterrows():
    
    if type(row['Cost']) == str:
        
        # Remove symbol
        if '£' in row['Cost']:
            dfcleancost.at[index, 'Cost'] = float(row['Cost'].replace("£", ""))
            
        # If it's a dollar amount, remove the dollar sign and calculate it in pounds
        if '$' in row ['Cost']:
            new = float(row['Cost'].replace("$", ""))
            new = round(new * .78, 2)
            dfcleancost.at[index, 'Cost'] = new
        
            
        

In [192]:
# Remove rows with null values as cost
dfcleancost = dfcleancost[pd.notnull(dfcleancost['Cost'])]


# Remove cost outliers, anything less than 50 and greater than 5000
#I should have removed these before the title sorting algorithm,  it would have lowered compile time :(

dfcleancost = dfcleancost.loc[dfcleancost['Cost'] <= 5000]

dfcleancost = dfcleancost.loc[dfcleancost['Cost'] >= 50]

In [195]:
dfcleancost.to_csv('AllClean.csv')



In [37]:
dfnew = pd.read_csv('AllClean.csv', encoding = "latin1")

In [51]:
dfgrouped = dfnew.groupby('Journal title').count()

In [52]:
# Found the 5 most common journal titles
dfgrouped.sort_values(by=['Unnamed: 0'], ascending = False).head(10)

,Unnamed: 0,PMID/PMCID,Publisher,Article title,Cost,Cleaned
Journal title,,,,,,
PloS one,198,188,198,198,198,198
The Journal of biological chemistry,60,59,60,60,60,60
Nucleic acids research,29,28,29,29,29,29
NeuroImage,29,28,29,29,29,29
PLoS pathogens,22,22,22,22,22,22
PLoS genetics,22,22,22,22,22,22
PLoS neglected tropical diseases,21,21,21,21,21,21
Proceedings of the National Academy of Sciences,19,19,19,19,19,19
Human molecular genetics,18,16,18,18,18,18


In [74]:
plosonedf = dfnew[dfnew['Journal title'] == 'PloS one']
biochemdf = dfnew[dfnew['Journal title'] == 'The Journal of biological chemistry']
neuleicdf = dfnew[dfnew['Journal title'] == 'Nucleic acids research']
neurodf = dfnew[dfnew['Journal title'] == 'NeuroImage']
pathogendf = dfnew[dfnew['Journal title'] == 'PLoS pathogens']
geneticdf = dfnew[dfnew['Journal title'] == 'PLoS genetics']

In [76]:
print('*****************************\nStatistics for PloS one\n*****************************')
print('Mean is: ' + str(plosonedf['Cost'].mean()))
print('Median is: ' + str(plosonedf['Cost'].median()))
print('Standard Deviation is: ' + str(plosonedf['Cost'].std()))

print('\n\n')

print('*****************************\nStatistics for The Journal of biological chemistry\n*****************************')
print('Mean is: ' + str(biochemdf['Cost'].mean()))
print('Median is: ' + str(biochemdf['Cost'].median()))
print('Standard Deviation is: ' + str(biochemdf['Cost'].std()))

print('\n\n')

print('*****************************\nStatistics for Nucleic acids research\n*****************************')
print('Mean is: ' + str(neuleicdf['Cost'].mean()))
print('Median is: ' + str(neuleicdf['Cost'].median()))
print('Standard Deviation is: ' + str(neuleicdf['Cost'].std()))

print('\n\n')

print('*****************************\nStatistics for NeuroImage\n*****************************')
print('Mean is: ' + str(neurodf['Cost'].mean()))
print('Median is: ' + str(neurodf['Cost'].median()))
print('Standard Deviation is: ' + str(neurodf['Cost'].std()))

print('\n\n')

print('*****************************\nStatistics for PLoS pathogens\n*****************************')
print('Mean is: ' + str(pathogendf['Cost'].mean()))
print('Median is: ' + str(pathogendf['Cost'].median()))
print('Standard Deviation is: ' + str(pathogendf['Cost'].std()))

print('\n\n')

print('*****************************\nStatistics for PLoS genetics\n*****************************')
print('Mean is: ' + str(geneticdf['Cost'].mean()))
print('Median is: ' + str(geneticdf['Cost'].median()))
print('Standard Deviation is: ' + str(geneticdf['Cost'].std()))

*****************************
Statistics for PloS one
*****************************
Mean is: 934.9821212121211
Median is: 896.975
Standard Deviation is: 194.96622876260946



*****************************
Statistics for The Journal of biological chemistry
*****************************
Mean is: 1407.925
Median is: 1313.13
Standard Deviation is: 391.00957391503897



*****************************
Statistics for Nucleic acids research
*****************************
Mean is: 1162.344827586207
Median is: 852.0
Standard Deviation is: 442.15093381769765



*****************************
Statistics for NeuroImage
*****************************
Mean is: 2215.168275862069
Median is: 2326.43
Standard Deviation is: 266.65394691928987



*****************************
Statistics for PLoS pathogens
*****************************
Mean is: 1481.385
Median is: 1439.225
Standard Deviation is: 241.41269729793973



*****************************
Statistics for PLoS geneticse
*****************************
Mean 